In [ ]:
!pip install torch==1.4.0

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
#! rm -r ./CodeXGLUE

In [ ]:
! pip install -q transformers==3.5.0 fastapi nest-asyncio pyngrok uvicorn
! pip install -U gdown

! git clone -q https://github.com/microsoft/CodeXGLUE.git

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
fatal: destination path 'CodeXGLUE' already exists and is not an empty directory.


In [ ]:

#cd ./CodeXGLUE/Code-Text/code-to-text/code

In [ ]:
import gdown

gdown.cached_download(
    "https://drive.google.com/uc?id=1CWcX7oT5xizjRpAwb6NcqHv4I1gQsaH8",
    "code_commenter_models.zip",
    postprocess=gdown.extractall,
)

File exists: code_commenter_models.zip


'code_commenter_models.zip'

In [ ]:
#!pip install torch.optim.lr_scheduler
import tensorflow

In [ ]:
! ls

code_commenter_models.zip  CodeXGLUE  model  sample_data


In [ ]:
import pathlib
pathlib.Path().resolve()

PosixPath('/content')

In [ ]:
from google.colab import files
src = list(files.upload().values())[0]
open('model1.py','wb').write(src)
import model1

Saving model.py to model.py


In [ ]:
from google.colab import files
src = list(files.upload().values())[0]
open('bleu1.py','wb').write(src)
import bleu1

Saving bleu.py to bleu.py


In [ ]:
from google.colab import files
src = list(files.upload().values())[0]
open('run1.py','wb').write(src)
import run1

Saving run.py to run (1).py


In [ ]:
! ls

code_commenter_models.zip  model      model.py	   sample_data
CodeXGLUE		   model1.py  __pycache__


In [ ]:
%%capture
import torch

import torch.nn as nn

from model1 import Seq2Seq
from pathlib import Path
from transformers import AutoTokenizer, RobertaConfig, RobertaModel

source_length = 256
target_length = 64
beam_size = 10
output_dir = f"model/java"
pretrained_model = "microsoft/codebert-base"

tokenizer = AutoTokenizer.from_pretrained(pretrained_model)
config = RobertaConfig.from_pretrained(pretrained_model)
encoder = RobertaModel.from_pretrained(pretrained_model, config = config)    
decoder_layer = nn.TransformerDecoderLayer(d_model=config.hidden_size, nhead=config.num_attention_heads)
decoder = nn.TransformerDecoder(decoder_layer, num_layers=6)
model = Seq2Seq(encoder = encoder,decoder = decoder,config=config,
                beam_size=beam_size,max_length=target_length,
                sos_id=tokenizer.cls_token_id,eos_id=tokenizer.sep_token_id)
model.load_state_dict(torch.load(Path(output_dir)/"checkpoint-best-bleu/pytorch_model.bin"))
model.to("cuda")

In [ ]:
from run1 import convert_examples_to_features, Example

class Args:
    max_source_length = source_length
    max_target_length = target_length

args = Args()

def prep_input(method):
    examples = [
        Example(0, source = method, target = "")
    ]
    eval_features = convert_examples_to_features(
        examples, tokenizer, args, stage="test"
    )
    source_ids = torch.tensor(
        eval_features[0].source_ids, dtype = torch.long
    ).unsqueeze(0).to("cuda")
    source_mask = torch.tensor(
        eval_features[0].source_mask, dtype = torch.long
    ).unsqueeze(0).to("cuda")

    return source_ids, source_mask

def gen_comment(method):
    source_ids, source_mask = prep_input(method)
    with torch.no_grad():
        pred = model(source_ids = source_ids, source_mask = source_mask)[0]
        t = pred[0].cpu().numpy()
        t = list(t)
        if 0 in t:
            t = t[:t.index(0)]
        text = tokenizer.decode(t,clean_up_tokenization_spaces=False)
        return text

In [ ]:
gen_comment("public static void checkbebra(Boolean checker){var i = 0; var v = 1; return i == v;}")

'tests bbra.'